In [1]:
## Each Space is a cell in a juptyr notebook ("QuickThinking" in fastai_workbench) that runs on Gradient Paperspace
from greek_normalisation.utils import (nfd, nfc, nfkc, convert_to_2019)
import numpy as np
import datetime
# from gensim.models import Phrases
from gensim.models import Word2Vec
from gensim.corpora import Dictionary

In [11]:
import pickle
import bz2
from greek_normalisation.utils import (nfd, nfc, nfkc, convert_to_2019)

def deserialize(mode_of_dataset):
    word2vec_documents_file = \
        bz2.BZ2File(f'{base_path}/{mode_of_dataset}_word2vec_documents_chrysostom.pickle', 'rb')
    lda_documents_file = \
        bz2.BZ2File(f'{base_path}/{mode_of_dataset}_lda_documents_chrysostom.pickle', 'rb')
    lda_dictionary_file = \
        bz2.BZ2File(f'{base_path}/{mode_of_dataset}_lda_dictionary_chrysostom.pickle', 'rb')
    lda_corpus_file = \
        bz2.BZ2File(f'{base_path}/{mode_of_dataset}_lda_corpus_chrysostom.pickle', 'rb')

    word2vec_documents = pickle.load(word2vec_documents_file)
    lda_documents = pickle.load(lda_documents_file)
    lda_dictionary = pickle.load(lda_dictionary_file)
    lda_corpus = pickle.load(lda_corpus_file)

    word2vec_documents_file.close()
    lda_documents_file.close()
    lda_dictionary_file.close()
    lda_corpus_file.close()

    return [word2vec_documents, lda_documents, lda_dictionary, lda_corpus]


# base_path = '/Users/paul/dev/fastai_workbench'
base_path = '/scratch365/psaieg/fastai_workbench'
mode_of_dataset = "training"
word2vec_documents, lda_documents, lda_dictionary, lda_corpus = \
    deserialize(mode_of_dataset)
# print(word2vec_documents )



In [12]:
#Train Word2Vec model
#https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-download-auto-examples-tutorials-run-word2vec-py

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import Word2Vec

word2vec_model = Word2Vec(
    sentences=word2vec_documents,
    min_count=10,
    size=200,
    workers=4, #requires Cython to be installed to have any effect. Othewise you are stuck on 1 core.
)


2021-02-07 01:02:21,022 : INFO : collecting all words and their counts
2021-02-07 01:02:21,023 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-07 01:02:21,136 : INFO : PROGRESS: at sentence #10000, processed 577501 words, keeping 23276 word types
2021-02-07 01:02:21,195 : INFO : collected 28292 word types from a corpus of 926260 raw words and 16223 sentences
2021-02-07 01:02:21,196 : INFO : Loading a fresh vocabulary
2021-02-07 01:02:21,218 : INFO : effective_min_count=10 retains 7339 unique words (25% of original 28292, drops 20953)
2021-02-07 01:02:21,219 : INFO : effective_min_count=10 leaves 877939 word corpus (94% of original 926260, drops 48321)
2021-02-07 01:02:21,237 : INFO : deleting the raw counts dictionary of 28292 items
2021-02-07 01:02:21,238 : INFO : sample=0.001 downsamples 32 most-common words
2021-02-07 01:02:21,239 : INFO : downsampling leaves estimated 841143 word corpus (95.8% of prior 877939)
2021-02-07 01:02:21,253 : INFO : esti

In [13]:
print(word2vec_model.most_similar_cosmul(nfc('λόγος')))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  """Entry point for launching an IPython kernel.
2021-02-07 01:02:33,984 : INFO : precomputing L2-norms of word weight vectors


[('τέος', 0.900982141494751), ('προσέχω', 0.8904626369476318), ('διόπερ', 0.8845375776290894), ('ἀποτείνω', 0.8802350759506226), ('ἀναγκαῖος', 0.8778597116470337), ('διδασκαλία', 0.8736364245414734), ('πειράω', 0.8713401556015015), ('εὐπαράδεκτος', 0.8702231049537659), ('ἐκκαλύπτω', 0.870146632194519), ('προοίμιον', 0.8695220351219177)]


In [14]:
# Train LDA model.
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import LdaModel

# Set training parameters.
num_topics = 50 #15
chunksize = 1500
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = lda_dictionary[0]  # This is only to "load" the lda_dictionary.
id2word = lda_dictionary.id2token

model = LdaModel(
    corpus = lda_corpus,
    id2word = id2word,
    chunksize = chunksize,
    alpha = 'auto',
    eta = 'auto',
    iterations = iterations,
    num_topics = num_topics,
    passes = passes,
    eval_every = eval_every
)

top_topics = model.top_topics(lda_corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
from pprint import pprint
pprint(top_topics)

2021-02-07 01:02:53,329 : INFO : using autotuned alpha, starting with [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
2021-02-07 01:02:53,333 : INFO : using serial LDA version on this node
2021-02-07 01:02:53,370 : INFO : running online (multi-pass) LDA training, 50 topics, 20 passes over the supplied corpus of 16223 documents, updating model once every 1500 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2021-02-07 01:02:53,377 : INFO : PROGRESS: pass 0, at document #1500/16223
2021-02-07 01:02:55,609 : INFO : optimized alpha [0.018867733, 0.018779825, 0.018606143, 0.018446622, 0.018388938, 0.01843223, 0.01854791, 0.018532615, 0.018853122, 0.01838868, 0.018664058, 0.019288056, 

2021-02-07 01:02:58,658 : INFO : topic #29 (0.023): 0.035*"θεός" + 0.014*"ὁράω" + 0.014*"ποιέω" + 0.011*"τιμωρία" + 0.011*"κύριος" + 0.010*"δεσπότης" + 0.009*"βούλομαι" + 0.009*"διάνοια" + 0.009*"μέλλω" + 0.008*"ἐπάγω"
2021-02-07 01:02:58,659 : INFO : topic #11 (0.024): 0.081*"θεός" + 0.032*"υἱός" + 0.013*"γῆ" + 0.010*"μανθάνω" + 0.010*"ποιέω" + 0.009*"οἶδα" + 0.008*"λόγος" + 0.008*"λέγω1" + 0.008*"δείκνυμι" + 0.007*"λαμβάνω"
2021-02-07 01:02:58,660 : INFO : topic diff=0.138978, rho=0.500000
2021-02-07 01:02:58,682 : INFO : PROGRESS: pass 0, at document #7500/16223
2021-02-07 01:02:59,612 : INFO : optimized alpha [0.022320462, 0.024049208, 0.021481892, 0.021986712, 0.0225414, 0.023008835, 0.020853644, 0.02242544, 0.023940573, 0.022351913, 0.02236993, 0.024757197, 0.021610504, 0.020511802, 0.023031505, 0.021695787, 0.02034129, 0.023417845, 0.021702807, 0.023212146, 0.0239745, 0.021648293, 0.022054264, 0.022788191, 0.02097218, 0.02243507, 0.021364398, 0.022187622, 0.022630008, 0.02416478

2021-02-07 01:03:03,094 : INFO : topic #8 (0.030): 0.024*"ποιέω" + 0.022*"ἰουδαῖος" + 0.020*"θεός" + 0.018*"νόμος" + 0.018*"κολάζω" + 0.017*"ἁμαρτάνω" + 0.016*"ἁμαρτία" + 0.015*"ἁμάρτημα" + 0.015*"τιμωρία" + 0.015*"ἀφίημι"
2021-02-07 01:03:03,095 : INFO : topic diff=0.164448, rho=0.353553
2021-02-07 01:03:03,118 : INFO : PROGRESS: pass 0, at document #13500/16223
2021-02-07 01:03:04,180 : INFO : optimized alpha [0.026696963, 0.027543427, 0.024467716, 0.025375279, 0.0267642, 0.025580369, 0.024590047, 0.025872689, 0.032445908, 0.025655547, 0.025764037, 0.030472357, 0.024497157, 0.023773562, 0.027003974, 0.025105843, 0.022865495, 0.027303716, 0.025969507, 0.027173644, 0.028887631, 0.025549153, 0.026473805, 0.02623198, 0.023888798, 0.024990141, 0.024713675, 0.025772832, 0.027764905, 0.028743554, 0.026715163, 0.024915371, 0.028027786, 0.025253072, 0.026202716, 0.02476399, 0.023765238, 0.025385512, 0.027225053, 0.027332943, 0.0266452, 0.024848856, 0.02410808, 0.023396987, 0.024318542, 0.024

2021-02-07 01:03:07,198 : INFO : topic diff=0.140174, rho=0.279341
2021-02-07 01:03:07,217 : INFO : PROGRESS: pass 1, at document #3000/16223
2021-02-07 01:03:08,221 : INFO : optimized alpha [0.029516157, 0.029268216, 0.0271343, 0.027627695, 0.02993684, 0.027121149, 0.0277452, 0.028166732, 0.038581375, 0.027800873, 0.02974537, 0.035704758, 0.027844056, 0.02675922, 0.030011604, 0.028073387, 0.024989475, 0.030497774, 0.028710637, 0.02937414, 0.031413328, 0.027504595, 0.029850066, 0.029261507, 0.026361024, 0.027388455, 0.027089272, 0.028172899, 0.031247312, 0.03145482, 0.028699437, 0.029168233, 0.031990737, 0.027507, 0.02870456, 0.028002, 0.02595835, 0.02858112, 0.03044323, 0.030157533, 0.029160451, 0.027978187, 0.025457073, 0.026738364, 0.02716441, 0.026900146, 0.028406851, 0.028552458, 0.029564818, 0.028750353]
2021-02-07 01:03:08,235 : INFO : merging changes from 1500 documents into a model of 16223 documents
2021-02-07 01:03:08,252 : INFO : topic #16 (0.025): 0.028*"σῶμα" + 0.020*"φιλ

2021-02-07 01:03:12,399 : INFO : optimized alpha [0.033472527, 0.033973288, 0.030067658, 0.031016147, 0.034310553, 0.030786386, 0.030230492, 0.03189319, 0.04498528, 0.03094111, 0.03324777, 0.04269686, 0.030450601, 0.02885031, 0.033925485, 0.031701047, 0.027359523, 0.035032533, 0.032586504, 0.033885363, 0.035994466, 0.031196807, 0.03316736, 0.03414163, 0.028857876, 0.03067498, 0.029739846, 0.03115437, 0.036031503, 0.037099168, 0.033956602, 0.032776408, 0.036024272, 0.030964501, 0.032532424, 0.030307058, 0.0287883, 0.032933544, 0.034659564, 0.03388044, 0.03262741, 0.030679904, 0.029200306, 0.029555831, 0.029875156, 0.030611277, 0.033129636, 0.031849183, 0.033348568, 0.032190427]
2021-02-07 01:03:12,412 : INFO : merging changes from 1500 documents into a model of 16223 documents
2021-02-07 01:03:12,429 : INFO : topic #16 (0.027): 0.026*"σῶμα" + 0.020*"ὕπνος" + 0.019*"μάχαιρα" + 0.018*"συνουσία" + 0.017*"ὑποδείκνυμι" + 0.016*"ὕγειος" + 0.015*"φιλέω" + 0.014*"χρόνος" + 0.013*"μέλος" + 0.013

...

2021-02-07 01:04:59,969 : INFO : topic diff=0.036437, rho=0.183139
2021-02-07 01:04:59,987 : INFO : PROGRESS: pass 18, at document #15000/16223
2021-02-07 01:05:00,291 : INFO : optimized alpha [1.1469926, 0.56978554, 0.19634223, 0.21486646, 0.26396537, 0.18131936, 0.27247503, 0.20024587, 1.2601614, 0.31642115, 0.7155148, 3.1016476, 0.16809645, 0.43146953, 0.3216342, 0.25669906, 0.19110009, 0.3472655, 0.31834352, 0.3585777, 0.27725327, 0.20199132, 0.43229425, 1.1198614, 0.22331902, 0.26409814, 0.3010221, 0.20391151, 0.49240568, 2.156615, 0.41999814, 0.28510436, 0.7989308, 0.26487285, 0.20821653, 0.28904644, 0.23790751, 0.2456503, 0.3300919, 0.26403597, 0.228679, 0.41585544, 0.3025664, 0.2194256, 0.3656676, 0.42434058, 0.2990627, 0.28493726, 4.3148565, 0.31054285]
2021-02-07 01:05:00,305 : INFO : merging changes from 1500 documents into a model of 16223 documents
2021-02-07 01:05:00,321 : INFO : topic #12 (0.168): 0.109*"ἰησοῦς" + 0.066*"χριστός" + 0.060*"πνεῦμα" + 0.056*"δόξα" + 0.056

2021-02-07 01:05:01,675 : INFO : merging changes from 1500 documents into a model of 16223 documents
2021-02-07 01:05:01,691 : INFO : topic #12 (0.174): 0.105*"ἰησοῦς" + 0.079*"χριστός" + 0.065*"πνεῦμα" + 0.055*"ἅγιος" + 0.054*"χάρις" + 0.051*"δόξα" + 0.049*"κύριος" + 0.043*"ἐπιτυγχάνω" + 0.038*"ἁμός" + 0.033*"φιλανθρωπία"
2021-02-07 01:05:01,692 : INFO : topic #5 (0.184): 0.077*"εὐχαριστέω" + 0.066*"ἀδὰμ" + 0.059*"λιμός" + 0.055*"πειρασμός" + 0.050*"ταπεινόω" + 0.048*"θλῖψις" + 0.038*"συντρίβω" + 0.027*"κινδυνεύω" + 0.025*"κυματόω" + 0.025*"ἰσραὴλ"
2021-02-07 01:05:01,693 : INFO : topic #29 (2.210): 0.046*"ποιέω" + 0.037*"τοσοῦτος" + 0.030*"ἐργάζομαι" + 0.029*"ἐπιδείκνυμι" + 0.022*"δεσπότης" + 0.017*"σωτηρία" + 0.017*"τυγχάνω" + 0.017*"οἰκεῖος" + 0.016*"βούλομαι" + 0.016*"ἐννοέω"
2021-02-07 01:05:01,694 : INFO : topic #11 (3.252): 0.133*"θεός" + 0.034*"λαμβάνω" + 0.027*"ποιέω" + 0.027*"κύριος" + 0.024*"καλέω" + 0.024*"υἱός" + 0.023*"ὁράω" + 0.017*"φύσις" + 0.014*"δείκνυμι" + 0.01

2021-02-07 01:05:03,088 : INFO : topic #5 (0.191): 0.076*"λιμός" + 0.069*"εὐχαριστέω" + 0.061*"πειρασμός" + 0.054*"θλῖψις" + 0.048*"ἀδὰμ" + 0.047*"ταπεινόω" + 0.037*"συντρίβω" + 0.029*"κινδυνεύω" + 0.027*"κυματόω" + 0.027*"ὑπομονὴν"
2021-02-07 01:05:03,089 : INFO : topic #29 (2.352): 0.046*"ποιέω" + 0.039*"τοσοῦτος" + 0.034*"ἐπιδείκνυμι" + 0.026*"ἐργάζομαι" + 0.024*"δεσπότης" + 0.018*"οἰκεῖος" + 0.017*"σωτηρία" + 0.016*"τυγχάνω" + 0.015*"βούλομαι" + 0.015*"ἐννοέω"
2021-02-07 01:05:03,090 : INFO : topic #11 (3.249): 0.131*"θεός" + 0.036*"λαμβάνω" + 0.029*"κύριος" + 0.027*"ποιέω" + 0.025*"καλέω" + 0.024*"ὁράω" + 0.021*"υἱός" + 0.020*"φύσις" + 0.014*"παῦλος" + 0.014*"χάρις"
2021-02-07 01:05:03,091 : INFO : topic #48 (4.369): 0.036*"ἀκούω" + 0.035*"λόγος" + 0.033*"οἶδα" + 0.032*"ποιέω" + 0.024*"βούλομαι" + 0.022*"ὁράω" + 0.021*"μανθάνω" + 0.019*"λέγω1" + 0.019*"ἔρχομαι" + 0.019*"δείκνυμι"
2021-02-07 01:05:03,092 : INFO : topic diff=0.037559, rho=0.180143
2021-02-07 01:05:03,110 : INFO 

2021-02-07 01:05:04,390 : INFO : topic #29 (2.362): 0.051*"ποιέω" + 0.035*"τοσοῦτος" + 0.029*"ἐπιδείκνυμι" + 0.027*"ἐργάζομαι" + 0.019*"δεσπότης" + 0.017*"ἐννοέω" + 0.016*"σωτηρία" + 0.015*"οἰκεῖος" + 0.015*"βούλομαι" + 0.014*"τυγχάνω"
2021-02-07 01:05:04,391 : INFO : topic #11 (3.436): 0.121*"θεός" + 0.038*"λαμβάνω" + 0.028*"ποιέω" + 0.027*"κύριος" + 0.026*"καλέω" + 0.024*"υἱός" + 0.022*"ὁράω" + 0.017*"φύσις" + 0.015*"δείκνυμι" + 0.013*"παῦλος"
2021-02-07 01:05:04,393 : INFO : topic #48 (4.816): 0.036*"ἀκούω" + 0.035*"λόγος" + 0.033*"ποιέω" + 0.031*"οἶδα" + 0.023*"βούλομαι" + 0.023*"δείκνυμι" + 0.022*"λέγω1" + 0.021*"ἔρχομαι" + 0.021*"ὁράω" + 0.018*"μανθάνω"
2021-02-07 01:05:04,394 : INFO : topic diff=0.041492, rho=0.180143
2021-02-07 01:05:04,452 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2021-02-07 01:05:04,478 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2021-02-07 01:05:04,505 : INFO : CorpusAccumulator accumulated stats from 3000 docume

Average topic coherence: -6.4581.
[([(0.10796549, 'ἰησοῦς'),
   (0.06886092, 'χριστός'),
   (0.06525057, 'πνεῦμα'),
   (0.054897133, 'χάρις'),
   (0.052106585, 'δόξα'),
   (0.05060135, 'ἐπιτυγχάνω'),
   (0.047998734, 'ἅγιος'),
   (0.044889398, 'κύριος'),
   (0.042817246, 'ἁμός'),
   (0.0387499, 'φιλανθρωπία'),
   (0.033776782, 'ομιλια'),
   (0.029032309, 'κύριος_ἰησοῦς'),
   (0.027146658, 'κράτος'),
   (0.021967715, 'ἰησοῦς_χριστός'),
   (0.020368606, 'χάρις_φιλανθρωπία'),
   (0.01956533, 'ἁμός_ομιλια'),
   (0.016662942, 'ἐπιτυγχάνω_χάρις'),
   (0.016487304, 'τιμή'),
   (0.015807604, 'φιλανθρωπία_κύριος'),
   (0.015144703, 'ἅγιος_πνεῦμα')],
  -1.8775150027240195),
 ([(0.03558117, 'ἀκούω'),
   (0.03535082, 'λόγος'),
   (0.033417873, 'ποιέω'),
   (0.030746592, 'οἶδα'),
   (0.022858905, 'βούλομαι'),
   (0.022823773, 'δείκνυμι'),
   (0.021541098, 'λέγω1'),
   (0.021243582, 'ἔρχομαι'),
   (0.020599958, 'ὁράω'),
   (0.017925102, 'μανθάνω'),
   (0.017627742, 'πρᾶγμα'),
   (0.015832782, '